In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# import xgboost as xgb

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,\
                             GradientBoostingClassifier, ExtraTreesClassifier,\
                             BaggingClassifier, VotingClassifier)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import KFold

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from time import time

C:\Users\Francisco\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Set de entrenamiento y Test final

Cargamos los archivos necesarios

In [148]:
train = pd.read_csv('csv_files/modelo_final.csv')
test_final = pd.read_csv('csv_files/test_final.csv')
#train = pd.read_csv('csv_files/modelo_final_featured.csv')
#test_final = pd.read_csv('csv_files/test_final_featured.csv')

Transformamos las variables categóricas al tipo correspondiente, para reducir el uso de memoria

In [149]:
train['rango_edad'] = train['rango_edad'].astype('category')
train['sexo'] = train['sexo'].astype('category') 
train['nivel_estudios'] = train['nivel_estudios'].astype('category')
train['esta_estudiando'] = train['esta_estudiando'].astype('category')
train['tipo_de_trabajo'] = train['tipo_de_trabajo'].astype('category')
train['nivel_laboral'] = train['nivel_laboral'].astype('category')
train['nombre_zona'] = train['nombre_zona'].astype('category')
train['sepostulo'] = train['sepostulo'].astype('category')

In [150]:
test_final['rango_edad'] = test_final['rango_edad'].astype('category')
test_final['sexo'] = test_final['sexo'].astype('category') 
test_final['nivel_estudios'] = test_final['nivel_estudios'].astype('category')
test_final['esta_estudiando'] = test_final['esta_estudiando'].astype('category')
test_final['tipo_de_trabajo'] = test_final['tipo_de_trabajo'].astype('category')
test_final['nivel_laboral'] = test_final['nivel_laboral'].astype('category')
test_final['nombre_zona'] = test_final['nombre_zona'].astype('category')

Nos guardamos los IDs necesarios para el submit

In [151]:
id_aviso_postulante = test_final['id']

Analizamos el set de entrenamiento y el test final

In [152]:
train.sample(5)

,idaviso,idpostulante,rango_edad,sexo,nivel_estudios,esta_estudiando,titulo,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area,sepostulo
288018,1112320305,xkd0kpY,1.0,1,2.0,0.0,Asistente geriátrica y auxiliar de enfermería,10,3,3,Salud,0.0
1808602,1112292828,rmdwz5O,1.0,2,2.0,0.0,Analista de Recursos Humanos - Formosa,10,3,3,Recursos Humanos,1.0
2148828,1112424032,W9P0RGX,3.0,2,4.0,1.0,"Asesores de Ventas de Seguros de Vida, Ahorro ...",10,3,3,Ventas,1.0
120430,1111455174,6rBYDGl,2.0,1,4.0,0.0,Vendedor viajante Resistencia Chaco.,10,3,3,Oficios y Profesiones,0.0
2994981,1112245051,EeYJaN,4.0,2,7.0,1.0,Ingeniero Proyecto - Administración de Contrat...,10,3,3,Control de Gestión,1.0


In [153]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 12 columns):
idaviso            int64
idpostulante       object
rango_edad         category
sexo               category
nivel_estudios     category
esta_estudiando    category
titulo             object
tipo_de_trabajo    category
nivel_laboral      category
nombre_zona        category
nombre_area        object
sepostulo          category
dtypes: category(8), int64(1), object(3)
memory usage: 114.4+ MB


In [154]:
test_final.sample(5)

,id,idaviso,idpostulante,nivel_estudios,esta_estudiando,sexo,rango_edad,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre_empresa
64446,64446,1112458235,0zBq4qM,4.0,1.0,1,1.0,Administrativo Contable,<p>Importante industria Nacional ubicada en La...,3.0,10.0,3.0,Administración,MAPSA
53311,53311,1112444844,E2Rb80,4.0,0.0,2,3.0,Analista Contable,<p>Prestigiosa Universidad se encuentra en la ...,3.0,10.0,2.0,Administración,Fundación Isalud
53550,53550,1112445282,8WJblL,4.0,0.0,1,2.0,ASISTENTE ADMINISTRATIVO Jr - Localiza Rent a Car,"<p>Localiza Rent a Car, rentadora de vehículos...",3.0,10.0,2.0,Administración,SERVIAUT SA
28953,28953,1112367849,ZWxZDZ,4.0,0.0,2,3.0,Liquidador de Sueldos,"<p style=""""><strong><u><span style="""">Liquidad...",3.0,10.0,3.0,Contabilidad,CARDINAL RRHH
18096,18096,1112208757,DrXOmbP,2.0,0.0,2,1.0,Repositor externo Zona Norte,<p><strong>Adecco Sales &amp; Marketing</stron...,3.0,10.0,3.0,Almacén / Depósito / Expedición,Adecco -Sales & Marketing


In [155]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
nivel_estudios     100000 non-null category
esta_estudiando    100000 non-null category
sexo               100000 non-null category
rango_edad         100000 non-null category
titulo             95552 non-null object
descripcion        95552 non-null object
nombre_zona        100000 non-null category
tipo_de_trabajo    100000 non-null category
nivel_laboral      100000 non-null category
nombre_area        95552 non-null object
nombre_empresa     95540 non-null object
dtypes: category(7), int64(2), object(5)
memory usage: 6.0+ MB


Nos guardamos los features que utilizaremos para los algoritmos de Machine Learning

In [156]:
# Features categóricos:
f1 = 'rango_edad'
f2 = 'sexo'
f3 = 'nivel_estudios'
f4 = 'esta_estudiando'
f5 = 'tipo_de_trabajo'
f6 = 'nivel_laboral'
f7 = 'nombre_zona'

f8 = 'estudios_vs_laboral'
f9 = 'estudia_vs_tipo_trabajo'
f10 = 'edad_vs_tipo_trabajo'
f11 = 'edad_vs_zona'

# Features strings:
'titulo'
'descripcion'
'nombre_area'
'nombre_empresa'

#features = [f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11]
features = [f1,f2,f3,f4,f5,f6,f7]
#features = [f8,f9,f10,f11]

Particionamos el set de entrenamiento para realizar pruebas locales de hiper-parámetros antes de realizar cada submit

In [157]:
test_s = 0.1
random_s = 0

x = np.array(train[features])
y = np.array(train['sepostulo'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_s, random_state=random_s)

Generamos el set de entrenamiento para el test final

In [158]:
x_test_final = np.array(test_final[features])

# Análisis de la Correlación de Pearson entre los distintos features

In [29]:
#colormap = plt.cm.RdBu
#plt.figure(figsize=(10,10))
#plt.title('Features - Correlación de Pearson', y=1.05, size=15)
#sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, 
#            square=True, cmap=colormap, linecolor='white', annot=True)


# Algoritmos probados

# 1. KNN

### Valores iniciales para K

In [ ]:
t0 = time()

print('VALORES INICIALES PARA K')

k_valores = [5,10,15]
mejor_k = 0
mejor_precision = 0

for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = accuracy_score(y_test, pred)
    print('La precisión para k=', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Valores grid search para K

In [ ]:
t0 = time()

print('VALORES GRID SEARCH PARA K')

k_valores = []

for i in range(mejor_k-2, mejor_k+3):
    k_valores.append(i)
    
for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = accuracy_score(y_test, pred)
    print('La precisión para k =', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Predecimos el test final con el mejor k obtenido

In [ ]:
#mejor_k = 

# Creamos el KNN
knn = KNeighborsClassifier(n_neighbors=mejor_k)

# Lo entrenamos con la totalidad del set de datos
knn.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = knn.predict(x_test_final)

### Submit File

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submits\submit_knn.csv', index=False)

submit['sepostulo'].value_counts()

# 2. Random Forest

### Set de entrenamiento

In [159]:
t0 = time()

# Parámetros óptimos
params_optimos_rf = {'n_estimators':100, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':5,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el Random Forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con nuestro set de entrenamiento
random_forest.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = random_forest.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.57179
Tiempo de ejecución: 202.23012 seconds.


### Test final

#### Predicciones con valores binarios

In [109]:
# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = random_forest.predict(x_test_final)

#### Predicciones con probabilidades

In [160]:
# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = random_forest.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

### Submit files

In [111]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_random_forest.csv', index=False)

In [112]:
submit['sepostulo'].value_counts()

0.0    54465
1.0    45535
Name: sepostulo, dtype: int64

In [161]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_random_forest_proba.csv', index=False)

In [162]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

0    59758
1    40242
Name: sepostulo, dtype: int64


# 3. Extra Trees

### Set de entrenamiento

In [114]:
t0 = time()

# Parámetros óptimos
params_optimos_et = {'n_estimators':100, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el extra_trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con nuestro set de entrenamiento
extra_trees.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = extra_trees.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.567796
Tiempo de ejecución: 37.07304 seconds.


### Test final

#### Predicciones con valores binarios

In [115]:
# Creamos el extra trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = extra_trees.predict(x_test_final)

#### Predicciones con probabilidades

In [116]:
# Creamos el extra trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = extra_trees.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

### Submit file

In [117]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})

submit.to_csv('submits/submit_extra_trees.csv', index=False)
submit_proba.to_csv('submits/submit_extra_trees_proba.csv', index=False)

In [118]:
submit['sepostulo'].value_counts()

0.0    63879
1.0    36121
Name: sepostulo, dtype: int64

In [119]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

0    59551
1    40449
Name: sepostulo, dtype: int64


# 4. Gaussian Naive Bayes

### Set de entrenamiento

In [55]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con nuestro set de entrenamiento
naive_bayes.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = naive_bayes.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.520864
Tiempo de ejecución: 1.01902 seconds.


### Test final

#### Predicciones con valores binarios

In [63]:
# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = naive_bayes.predict(x_test_final)

#### Predicciones con probabilidades

In [64]:
# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = naive_bayes.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

### Submit file

In [65]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})

submit.to_csv('submits/submit_naive_bayes.csv', index=False)
submit_proba.to_csv('submits/submit_naive_bayes_proba.csv', index=False)

In [66]:
submit['sepostulo'].value_counts()

1    59833
0    40167
Name: sepostulo, dtype: int64

In [67]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

0    40167
1    59833
Name: sepostulo, dtype: int64


# 5. Perceptron

### Set de entrenamiento

In [58]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con nuestro set de entrenamiento
perceptron.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = perceptron.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.500252
Tiempo de ejecución: 1.45909 seconds.


### Test final

#### Predicciones con valores binarios

In [68]:
# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con la totalidad del set de datos
perceptron.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = perceptron.predict(x_test_final)

### Submit file

In [70]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submits\submit_perceptron.csv', index=False)

submit['sepostulo'].value_counts()

0    100000
Name: sepostulo, dtype: int64

# 6. SVM

### Set de entrenamiento

In [ ]:
t0 = time()

# Parámetros óptimos
params_optimos_svm = {'C':1.0, 'kernel':'rbf', 'gamma':'auto'}

# Creamos el svm
svm = SVC(**params_optimos_svm)

# Lo entrenamos con nuestro set de entrenamiento
svm.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = svm.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

params_optimos_svm = {'C':1.0, 'kernel':'rbf', 'gamma':'auto'}

# Creamos el svm
svm = SVC(**params_optimos_svm)

# Lo entrenamos con la totalidad del set de datos
svm.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = svm.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submits\submit_svm.csv', index=False)

submit['sepostulo'].value_counts()

# 7. Adaptive Boosting

### Set de entrenamiento

In [71]:
t0 = time()

# Parámetros óptimos
params_optimos_ab = {'n_estimators':50}

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con nuestro set de entrenamiento
ada_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = ada_boost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.529852
Tiempo de ejecución: 26.84970 seconds.


### Test final

#### Predicciones con valores binarios

In [72]:
# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = ada_boost.predict(x_test_final)

#### Predicciones con probabilidades

In [75]:
# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = ada_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

### Submit file

In [76]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})

submit.to_csv('submits/submit_ada_boost.csv', index=False)
submit_proba.to_csv('submits/submit_ada_boost_proba.csv', index=False)

In [77]:
submit['sepostulo'].value_counts()

1    50479
0    49521
Name: sepostulo, dtype: int64

In [78]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

0    49521
1    50479
Name: sepostulo, dtype: int64


# 8. Gradient Boosting

### Set de entrenamiento

In [121]:
t0 = time()

# Parámetros óptimos
params_optimos_gb = {'learning_rate':0.1, 'n_estimators':75, 'max_depth':4, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'subsample':1.0, 'max_features':'sqrt', 'warm_start':True}

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con nuestro set de entrenamiento
gra_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = gra_boost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.579472
Tiempo de ejecución: 32.61337 seconds.


### Test final

#### Predicciones con valores binarios

In [122]:
# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = gra_boost.predict(x_test_final)

#### Predicciones con probabilidades

In [123]:
# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = gra_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

### Submit file

In [124]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})

submit.to_csv('submits/submit_gra_boost.csv', index=False)
submit_proba.to_csv('submits/submit_gra_boost_proba.csv', index=False)

In [125]:
submit['sepostulo'].value_counts()

0.0    62205
1.0    37795
Name: sepostulo, dtype: int64

In [126]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

0    64084
1    35916
Name: sepostulo, dtype: int64


# 9. Logistic Regression

### Set de entrenamiento

In [14]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con nuestro set de entrenamiento
log_reg.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = log_reg.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.522424
Tiempo de ejecución: 3.97218 seconds.


### Test final

#### Predicciones con valores binarios

In [85]:
# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = log_reg.predict(x_test_final)

#### Predicciones con probabilidades

In [86]:
# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = log_reg.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

### Submit file

In [87]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})

submit.to_csv('submits/submit_log_reg.csv', index=False)
submit_proba.to_csv('submits/submit_log_reg_proba.csv', index=False)

In [88]:
submit['sepostulo'].value_counts()

0    56324
1    43676
Name: sepostulo, dtype: int64

In [89]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

0    56324
1    43676
Name: sepostulo, dtype: int64


# 10. Decision Tree

### Set de entrenamiento

In [127]:
t0 = time()

# Parámetros óptimos
params_optimos_dt = {'criterion':'entropy', 'splitter':'best', 'max_depth':None, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'max_features':'sqrt', 'presort':True}

# Creamos el logistic regression
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con nuestro set de entrenamiento
decision_tree.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = decision_tree.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.583244
Tiempo de ejecución: 12.45123 seconds.


### Test final

#### Predicciones con valores binarios

In [128]:
# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = decision_tree.predict(x_test_final)

#### Predicciones con probabilidades

In [129]:
# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = decision_tree.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

### Submit file

In [130]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})

submit.to_csv('submits/submit_decision_tree.csv', index=False)
submit_proba.to_csv('submits/submit_decision_tree_proba.csv', index=False)

In [131]:
submit['sepostulo'].value_counts()

0.0    59439
1.0    40561
Name: sepostulo, dtype: int64

In [132]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

0    59821
1    40179
Name: sepostulo, dtype: int64


# 11. Bagging

## 11.1 Con Logistic Regression

### Set de entrenamiento

In [24]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                  'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.522968
Tiempo de ejecución: 243.49726 seconds.


### Test final

#### Predicciones con valores binarios

In [25]:
# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

#### Predicciones con probabilidades

### Submit file

In [26]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submits\submit_bagging_log_reg.csv', index=False)

submit['sepostulo'].value_counts()

0    56350
1    43650
Name: sepostulo, dtype: int64

## 11.2 Con Decision Tree

### Set de entrenamiento

In [27]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_dt = {'base_estimator':DecisionTreeClassifier(**params_optimos_dt), 'n_estimators':10, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.53406
Tiempo de ejecución: 19.59580 seconds.


### Test final

#### Predicciones con valores binarios

In [28]:
# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

#### Predicciones con probabilidades

### Submit file

In [30]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submits\submit_bagging_decision_tree.csv', index=False)

submit['sepostulo'].value_counts()

0    50858
1    49142
Name: sepostulo, dtype: int64

# 12. Voting

### Set de entrenamiento

In [34]:
# Creamos los estimadores base que utilizaremos para el voting
estimador_1 = DecisionTreeClassifier(**params_optimos_dt)
estimador_2 = GradientBoostingClassifier(**params_optimos_gb)
estimador_3 = ExtraTreesClassifier(**params_optimos_et)

# Los agregamos a la lista de estimadores
estimadores = []
estimadores.append(('Naive Bayes', estimador_1))
estimadores.append(('Logistic Regression', estimador_2))
estimadores.append(('Random Forest', estimador_3))

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con nuestro set de entrenamiento
voting.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = voting.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.534204
Tiempo de ejecución: 165.94960 seconds.


C:\Users\Francisco\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Test final

#### Predicciones con valores binarios

In [35]:
# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con la totalidad del set de datos
voting.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = voting.predict(x_test_final)

C:\Users\Francisco\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Predicciones con probabilidades

### Submit file

In [36]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submits\submit_voting.csv', index=False)

submit['sepostulo'].value_counts()

0    50485
1    49515
Name: sepostulo, dtype: int64

# 13. Ensamble

## Definición de algunas variables globales

In [ ]:
ntrain = train.shape[0] # Cantidad de registros totales del set de entrenamiento

ntest = test_final.shape[0] # Cantidad de registros totales del test final

SEED = 0 # Seed

NFOLDS = 4 # Cantidad de bloques a particionar para KFold

kfold = KFold(ntrain, n_folds=NFOLDS, random_state=SEED) # KFold

## Creación de la clase SklearnPadre

Lo que realizamos a continuación es la creación de una clase que posee los métodos comunes de los clasificadores de Sklearn que utilizaremos en el presente trabajo. De esta forma, ahorramos líneas de código y evitamos redundancia a la hora de utilizar los métodos.

El inicializador recibe un clasificador de Sklearn, un seed y los parámetros necesarios del clasificador en cuestión.

In [ ]:
class SklearnPadre(object):
    def __init__(self, clasif, seed=0, parametros=None):
        parametros['random_state'] = seed
        self.clasif = clasif(**parametros)

    def train(self, x_train, y_train):
        self.clasif.fit(x_train, y_train)

    def predict(self, x):
        return self.clasif.predict(x)
    
    def fit(self, x, y):
        return self.clasif.fit(x,y)
    
    def feature_importances(self, x, y):
        print(self.clasif.fit(x,y).feature_importances_)

## Predicciones OOF (Out Of Fold)

El ensamble utiliza predicciones de los clasificadores base como input para el entrenamiento del modelo de 2do nivel. Sin embargo, no podemos entrenar los modelos base con la totalidad del set de entrenamiento. Esto genera el riesgo de que nuestro modelo de 1er nivel conozca el test final y overfitee cuando utilice las predicciones base.

Por ello, utilizamos la siguiente función para tomar distintos bloques del set de entrenamiento.

In [ ]:
def get_oof(clasif, x_train, y_train, x_test):
    oof_train = np.zeros(ntrain)
    oof_test = np.zeros(ntest)
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kfold):
        x_train_i = x_train[train_index]
        y_train_i = y_train[train_index]
        x_test_i = x_train[test_index]

        clasif.train(x_train_i, y_train_i)

        oof_train[test_index] = clasif.predict(x_test_i)
        oof_test_skf[i, :] = clasif.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1) 

## Modelo de 1er Nivel

A continuación llevaremos a cabo nuestro primer nivel de clasificación mediante el uso de algunos clasificadores de la librería Sklearn:

- Random Forest
- Extra Trees
- Decision Tree
- Gradient Boosting
- TO DO: elegir alguno

### Definición de los hiper-parámetros necesarios para cada algoritmo

In [ ]:
# RANDOM FOREST
rf_params = {'n_estimators':50, 'max_features':'sqrt', 'max_depth':15, 'min_samples_split':5,\
             'min_samples_leaf':2, 'bootstrap':True, 'oob_score':False, 'warm_start':True}

# EXTRA TREES
et_params = {'n_estimators':100, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
             'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}


# DECISION TREE
dt_params = {'criterion':'entropy', 'splitter':'best', 'max_depth':None, 'min_samples_split':2,\
             'min_samples_leaf':2, 'max_features':'sqrt', 'presort':True}


# GRADIENT BOOSTING
gb_params = {'learning_rate':0.1, 'n_estimators':75, 'max_depth':4, 'min_samples_split':2,\
             'min_samples_leaf':2, 'subsample':1.0, 'max_features':'sqrt', 'warm_start':True}

# TO DO: elegir alguno

### Creación de los clasificadores según los hiper-parámetros definidos

In [ ]:
random_forest = SklearnPadre(clasif=RandomForestClassifier, seed=SEED, parametros=rf_params)

extra_trees = SklearnPadre(clasif=ExtraTreesClassifier, seed=SEED, parametros=et_params)

decision_tree = SklearnPadre(clasif=DecisionTreeClassifier, seed=SEED, parametros=dt_params)

grad_boost = SklearnPadre(clasif=GradientBoostingClassifier, seed=SEED, parametros=gb_params)

# TO DO: elegir alguno

### Output de los modelos de primer nivel

A continuación realizamos las predicciones para el set de entrenamiento y el test final con OOF. Estos resultados base serán utilizados como input en el modelo de segundo nivel.

In [ ]:
t0_total = time()

# Random Forest
t0_rf = time()
rf_oof_train, rf_oof_test = get_oof(random_forest, x, y, x_test_final)
tf_rf = time() - t0_rf
print ("Tiempo de entrenamiento del random forest: %0.5f seconds." % tf_rf)

# Extra Trees
t0_et = time()
et_oof_train, et_oof_test = get_oof(extra_trees, x, y, x_test_final)
tf_et = time() - t0_et
print ("Tiempo de entrenamiento del extra trees: %0.5f seconds." % tf_et)

# Decision Tree
t0_dt = time()
dt_oof_train, dt_oof_test = get_oof(decision_tree, x, y, x_test_final)
tf_dt = time() - t0_dt
print ("Tiempo de entrenamiento del decision tree: %0.5f seconds." % tf_dt)

# Gradient Boosting
t0_gb = time()
gb_oof_train, gb_oof_test = get_oof(grad_boost, x, y, x_test_final)
tf_gb = time() - t0_gb
print ("Tiempo de entrenamiento del gradient boosting: %0.5f seconds." % tf_gb)

# TO DO: elegir alguno
t0_svm = time()
svm_oof_train, svm_oof_test = get_oof(svm, x, y, x_test_final)
tf_svm = time() - t0_svm
print ("Tiempo de entrenamiento del SVM: %0.5f seconds." % tf_svm)

# Tiempo total de ejecución
tf_total = time() - t0_total
print ("Tiempo total de entrenamiento del modelo de 1er nivel: %0.5f seconds." % tf_total)

### Importancia de los features en base a los resultados obtenidos

In [ ]:
rf_features = random_forest.feature_importances(x,y)

et_features = extra_trees.feature_importances(x,y)

dt_features = decision_tree.feature_importances(x,y)

gb_features = grad_boost.feature_importances(x,y)

# TO DO: elegir alguno
#svm_features = svm.feature_importances(x,y)

In [ ]:
# ACA HAY QUE COPIAR Y PEGAR LA SALIDA DE ARRIBA PERO EN FORMATO ARRAY

### Creación de un dataframe con la importancia de cada feature para cada clasificador

In [ ]:
features = np.array(features)

df_features = pd.DataFrame({'Features': features,\
                            'Random Forest': rf_features,\
                            'Extra Trees': et_features,\
                            'Decision Tree': dt_features,\
                            'Gradient Boost': gb_features,\
     # TO DO: elegir alguno                       'SVM': svm_features\
                            })

df_features = df_features[['Features', 'Random Forest', 'Extra Trees', 'Decision Tree', 'Gradient Boost', 'SVM']]

df_features.head()

In [ ]:
# TO DO: plotear el df de arriba. Podría ser un heatmap

Agregamos una columna contenedora del promedio de la importancia del feature para todos los clasificadores.

In [ ]:
df_features['promedio'] = df_features.mean(axis=1)
df_features.head()

## Modelo de 2do Nivel

Utilizamos las predicciones obtenidas en el modelo de primer nivel como input del siguiente modelo.

Primero, analizamos las predicciones del primer nivel para cada clasificador.

In [ ]:
predicciones_primer_nivel = pd.DataFrame({'Random Forest': rf_oof_train.ravel(),\
                                          'Extra Trees': et_oof_train.ravel(),\
                                          'Decision Tree': dt_oof_train.ravel(),\
                                          'Gradient Boosting': gb_oof_train.ravel(),\
 # TO DO: elegir alguno                                         'SVM':svm_oof_train.ravel()\
                                          })
predicciones_primer_nivel.head()

### Correlación de los clasificadores del modelo de 1er nivel (set de entrenamiento del modelo de 2do nivel)

In [ ]:
# TO DO: tirar un heatmap con predicciones_primer_nivel.corr()

### Concatenamos las predicciones obtenidas en el modelo de 1er nivel

In [ ]:
x_2do_nivel = np.concatenate((et_oof_train, rf_oof_train, ab_oof_train, gb_oof_train, svm_oof_train), axis=1)

x_test_final_2do_nivel = np.concatenate((et_oof_test, rf_oof_test, ab_oof_test, gb_oof_test, svm_oof_test), axis=1)

### Aplicamos XGBoost para el modelo de 2do nivel

#### Predicciones con valores binarios

In [ ]:
# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\                        
                            subsample=0.7,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con el set de entrenamiento obtenido en el modelo de 1er nivel
xgboost.fit(x_2do_nivel, y)

# Realizamos las predicciones con el set definitivo
pred_final = gbm.predict(x_test_final_2do_nivel)

#### Predicciones con probabilidades

## Submit File

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submits\submit_ensamble.csv', index=False)

submit['sepostulo'].value_counts()